# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [225]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest as pz

In [199]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


## 1. What test is appropriate for this problem? 

- Does CLT apply?

In [52]:
#
# Print the data info.
#
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

In [181]:
#
# Perform an independence test.
#
stats.chi2_contingency(data.groupby('race').sum())


(43.10592882275931,
 0.85613063138116319,
 54,
 array([[  8.81572865e+03,   8.92028473e+03,   1.91077492e+04,
           1.28568963e+02,   1.00253775e+03,   2.36626924e+02,
           1.09158551e+03,   5.25359797e+05,   8.48204967e+03,
           1.36323122e+03,   1.16762630e+03,   1.99907228e+03,
           8.00929609e+02,   1.22365635e+03,   1.21265045e+03,
           1.96105188e+02,   1.58793075e+06,   7.43901186e+02,
           1.29899767e+03,   2.42850907e+04,   4.44365565e+02,
           5.11727182e+02,   2.28577841e+04,   1.75294026e+03,
           7.09380503e+02,   4.91569578e+05,   1.97020805e+06,
           5.96313840e+04,   2.48665584e+05,   1.78095528e+02,
           7.58933236e+01,   8.09601985e+02,   1.01709741e+04,
           9.75749439e+01,   3.20355856e+02,   9.62537582e+03,
           3.70698838e+02,   1.88100895e+02,   8.10934976e+02,
           2.89155099e+02,   3.68197497e+02,   4.09219500e+02,
           1.91802881e+03,   1.06056888e+03,   3.04163149e+02,
        

### Result

- CLT does apply as the observations are independent and the sample size is sufficiently large to assume a normal distribution.
- A Chi-squared test is appropriate for this problem.

## 2. What are the null and alternate hypotheses?


### Answer:

- The null hypothesis is that probability of a callback for 'b' is equal to 'w' or p_b = p_w.
- The alternate hypothesis is p_b≠p_w

## 3. Compute margin of error, confidence interval, and p-value.

In [217]:
#
# Split the data into 'w' and 'b'.
#
data_w = data.call[data.race == 'w']
data_b = data.call[data.race == 'b']
num_b = len(data_b)
num_w = len(data_w)
print("Observations where race is b: ", num_b)
print("Observations where race is w: ", num_w)


Observations where race is b:  2435
Observations where race is w:  2435


In [219]:
#
# Calculate and print the successes.
#
b_success = len(data_b[data_b == 1])
w_success = len(data_w[data_w == 1])
print(b_success, w_success)

157 235


In [221]:
#
# Calculate the proportion of black sounding names getting a callback and 
# the proportion white sounding names getting a callback.
#
p_b = 1.0 * b_success / num_b
p_w = 1.0 * w_success / num_w
print("Proportion of black sounding names getting a callback: ", p_b)
print("Proportion of white sounding names getting a callback: ", p_w)

Proportion of black sounding names getting a callback:  0.06447638603696099
Proportion of white sounding names getting a callback:  0.09650924024640657


In [223]:
#
# Determine the 'b' and 'w' callback probabilities.
#
z = 2.58
p_w = np.sum(data_w == 1) / len(data_w)

#
# Calculate p_pool
#
p_pool = (np.sum(data_w) + np.sum(data_b)) / (len(data_w) + len(data_b))

#
# Calcualate the standard error.
#
se = np.sqrt(((p_pool * (1 - p_pool)) / len(data_w)) + ((p_pool * (1 - p_pool)) / len(data_b)))

#
# Calculate the margin of error for a 99% confidence interval.
#
me = z * se
print(me)


0.0201159866133


In [233]:
#
# Calculate the confidence interval.
#
interval = [p_w - p_b - me, p_w - p_b + me]
print(interval)

[0.011916867596125806, 0.052148840822765365]


In [228]:
#
# Calculate and print the p-value.
#
p_val = pz(np.array([b_success, w_success]), np.array([num_b, num_w]), value=0)[1]
print(p_val)

3.98388683759e-05


### Result

- 0 is not included in the 99% confidence interval.  This means that there is a 99% probability that the true population proportion is 0.032±0.02.  A p-value of 3.98388683759e-05 tells us that we can reject the null hypothesis.

## 4. Write a story describing the statistical significance in the context or the original problem.


### Summary

- The original problem was to determine whether race has a significant impact on the rate of callbacks for resumes.  From the tests above, it is clear that a black sounding name negativetly influences whether or not that person receives a callback. 

## 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

### No, the analysis does not mean that race/name is the most important factor in callback success.  All it means is that race, plays enough of a role to be statistically significant.  Said another way, race has some influence on the outcome of the callback variable.  More data is needed to determine if/what other factors may be influencing the outcome.